In [0]:
import torch
from torch import nn
import torchvision as tv
from torchvision import datasets, models, transforms
import time

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls drive/My\ Drive/road_sign

test  train  TsignRecgTest1994Annotation.txt  TsignRecgTrain4170Annotation.txt


In [0]:
data_transform = transforms.Compose([
        transforms.Resize((224,224)),                             
        transforms.RandomResizedCrop((224,224), scale=(0.8,1), ratio=(1,1)),
        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
data_transform_32 = transforms.Compose([
        transforms.Resize((32,32)),                             
       # transforms.RandomResizedCrop(28, scale=(0.9,1), ratio=(0.9,1.1)),
        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

road_sign_dataset_train = datasets.ImageFolder(root='drive/My Drive/road_sign/train',
                                           transform=data_transform)
dataset_train_loader = torch.utils.data.DataLoader(road_sign_dataset_train,
                                             batch_size=32, shuffle=True)

road_sign_dataset_test = datasets.ImageFolder(root='drive/My Drive/road_sign/test',
                                           transform=data_transform)
dataset_test_loader = torch.utils.data.DataLoader(road_sign_dataset_test,
                                             batch_size=32, shuffle=True)


road_sign_dataset_train_32 = datasets.ImageFolder(root='drive/My Drive/road_sign/train',
                                           transform=data_transform_32)
dataset_train_loader_32 = torch.utils.data.DataLoader(road_sign_dataset_train_32,
                                             batch_size=64, shuffle=True)

road_sign_dataset_test_32 = datasets.ImageFolder(root='drive/My Drive/road_sign/test',
                                           transform=data_transform_32)
dataset_test_loader_32 = torch.utils.data.DataLoader(road_sign_dataset_test_32,
                                             batch_size=64, shuffle=True)

In [0]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 12, kernel_size=3),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=3, stride=2),
    torch.nn.Conv2d(12, 36, kernel_size=2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2),
    torch.nn.Flatten(),
    torch.nn.Linear(1296, 256),
    torch.nn.BatchNorm1d(256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 58)
)

In [0]:
for X, y in dataset_train_loader_28:
    break

In [0]:
for X, y in dataset_train_loader_32:
   print(X.shape,y.shape)
   break

torch.Size([4, 3, 28, 28]) torch.Size([4])


In [0]:
X_ = X
for m in model:
  X_ = m(X_)
  print(X_.shape)

torch.Size([4, 18, 28, 28])
torch.Size([4, 18, 28, 28])
torch.Size([4, 18, 14, 14])
torch.Size([4, 36, 10, 10])
torch.Size([4, 36, 10, 10])
torch.Size([4, 36, 5, 5])
torch.Size([4, 900])
torch.Size([4, 600])
torch.Size([4, 600])
torch.Size([4, 58])


## Модель и обучение

In [0]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [0]:
dev 

device(type='cuda')

In [0]:
model = model.to(dev)

In [0]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [0]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [0]:
lr, num_epochs = 0.005, 20
trainer = torch.optim.AdamW(model.parameters(), lr=lr)
train(model, dataset_train_loader_32, dataset_test_loader_32, trainer, num_epochs, dev)

epoch 1, loss 1.0103, train acc 0.759, test acc 0.225, time 1431.0 sec
epoch 2, loss 0.1200, train acc 0.974, test acc 0.227, time 15.3 sec
epoch 3, loss 0.0201, train acc 0.997, test acc 0.231, time 14.8 sec
epoch 4, loss 0.0057, train acc 1.000, test acc 0.229, time 14.7 sec
epoch 5, loss 0.0024, train acc 1.000, test acc 0.228, time 15.3 sec
epoch 6, loss 0.0010, train acc 1.000, test acc 0.232, time 14.6 sec
epoch 7, loss 0.0009, train acc 1.000, test acc 0.233, time 14.5 sec
epoch 8, loss 0.0008, train acc 1.000, test acc 0.230, time 14.4 sec
epoch 9, loss 0.0005, train acc 1.000, test acc 0.232, time 14.5 sec
epoch 10, loss 0.0004, train acc 1.000, test acc 0.232, time 14.3 sec
epoch 11, loss 0.0004, train acc 1.000, test acc 0.233, time 14.3 sec
epoch 12, loss 0.0004, train acc 1.000, test acc 0.234, time 14.2 sec
epoch 13, loss 0.0003, train acc 1.000, test acc 0.234, time 14.2 sec
epoch 14, loss 0.0003, train acc 1.000, test acc 0.232, time 14.1 sec
epoch 15, loss 0.0002, trai

In [0]:
# Попробуем резнет50

In [0]:
model_ft = models.resnet50(pretrained=True)

num_ftrs = model_ft.fc.in_features

for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs

2048

In [0]:
model_ft.fc = nn.Sequential(
   
    nn.Linear(num_ftrs, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.1),

    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.1),

    nn.Linear(512, 256),
    nn.ReLU(),
    nn.BatchNorm1d(256),
    nn.Dropout(0.1),
    
    nn.Linear(256, 58)
)    



In [0]:
for X, y in dataset_train_loader:
   print(X.shape,y.shape)
   break

torch.Size([4, 3, 224, 224]) torch.Size([4])


In [0]:
model_ft = model_ft.to(dev)

In [0]:
model_ft.layer4.parameters()

<generator object Module.parameters at 0x7fac965a2780>

In [0]:
lr, num_epochs = 0.01, 20
trainer = torch.optim.Adamax(model_ft.parameters(), lr=lr)
train(model_ft, dataset_train_loader, dataset_test_loader, trainer, num_epochs, dev)

epoch 1, loss 1.1163, train acc 0.703, test acc 0.139, time 39.5 sec
epoch 2, loss 0.3983, train acc 0.879, test acc 0.130, time 39.7 sec
epoch 3, loss 0.2377, train acc 0.926, test acc 0.149, time 39.4 sec
epoch 4, loss 0.1586, train acc 0.949, test acc 0.136, time 39.7 sec
epoch 5, loss 0.1303, train acc 0.963, test acc 0.142, time 39.6 sec
epoch 6, loss 0.1233, train acc 0.958, test acc 0.154, time 39.9 sec
epoch 7, loss 0.1308, train acc 0.957, test acc 0.141, time 39.7 sec
epoch 8, loss 0.0977, train acc 0.969, test acc 0.151, time 40.1 sec
epoch 9, loss 0.0977, train acc 0.970, test acc 0.148, time 39.7 sec
epoch 10, loss 0.0934, train acc 0.969, test acc 0.148, time 39.9 sec
epoch 11, loss 0.0694, train acc 0.978, test acc 0.158, time 39.8 sec
epoch 12, loss 0.0632, train acc 0.977, test acc 0.160, time 39.7 sec
epoch 13, loss 0.0676, train acc 0.980, test acc 0.163, time 39.3 sec
epoch 14, loss 0.0624, train acc 0.979, test acc 0.139, time 40.4 sec
epoch 15, loss 0.0785, train 

In [0]:
lr, num_epochs = 0.001, 5
for param in model_ft.layer4.parameters():
    param.requires_grad = True
    
trainer = torch.optim.SGD([
                {'params': model_ft.layer4.parameters(), 'lr': lr/100},
                {'params': model_ft.fc.parameters()}
            ], lr=lr, momentum=0.9)
train(model_ft, dataset_train_loader, dataset_test_loader, trainer, num_epochs, dev)

epoch 1, loss 0.0171, train acc 0.995, test acc 0.170, time 43.6 sec
epoch 2, loss 0.0126, train acc 0.996, test acc 0.175, time 43.8 sec
epoch 3, loss 0.0141, train acc 0.994, test acc 0.175, time 44.2 sec
epoch 4, loss 0.0109, train acc 0.997, test acc 0.163, time 44.0 sec
epoch 5, loss 0.0078, train acc 0.998, test acc 0.164, time 44.3 sec


In [0]:
lr, num_epochs = 0.0001, 10
trainer = torch.optim.SGD([
                {'params': model_ft.layer4.parameters(), 'lr': lr/100},
                {'params': model_ft.fc.parameters()}
            ], lr=lr, momentum=0.9)
train(model_ft, dataset_train_loader, dataset_test_loader, trainer, num_epochs, dev)

epoch 1, loss 0.0031, train acc 0.999, test acc 0.158, time 44.5 sec
epoch 2, loss 0.0066, train acc 0.998, test acc 0.159, time 44.2 sec
epoch 3, loss 0.0056, train acc 0.999, test acc 0.164, time 44.2 sec
epoch 4, loss 0.0035, train acc 1.000, test acc 0.169, time 44.4 sec
epoch 5, loss 0.0049, train acc 0.999, test acc 0.170, time 44.7 sec
epoch 6, loss 0.0037, train acc 0.999, test acc 0.163, time 44.9 sec
epoch 7, loss 0.0039, train acc 1.000, test acc 0.165, time 45.2 sec
epoch 8, loss 0.0039, train acc 0.999, test acc 0.169, time 44.4 sec
epoch 9, loss 0.0027, train acc 0.999, test acc 0.162, time 44.3 sec
epoch 10, loss 0.0031, train acc 0.999, test acc 0.170, time 44.4 sec


In [0]:
# Возможная причина низкого скора на тесте дисбаланс классов